# Welding Anomaly Detection with ResNet-50 Binary Classifier

This notebook demonstrates a complete pipeline for welding anomaly detection using:
1. **YOLO** for weld region detection
2. **ResNet-50** fine-tuned as a binary classifier for good/bad weld classification

## Data Split Strategy
- **Training Set**: Used for model training and parameter updates
- **Validation Set**: Used for model selection (early stopping) AND final evaluation
- **No Test Set**: Validation set serves dual purpose for simplicity

## Pipeline Overview
- Extract weld ROIs from images using XML annotations
- Create a balanced dataset with proper train/val splits  
- Fine-tune ResNet-50 with careful data preprocessing and augmentation
- Evaluate using ROC-AUC, PR-AUC, and F1 metrics on validation set

⚠️ **Note**: In production, consider using a separate test set for unbiased final evaluation.

In [ ]:
!pip install -r cnn_train/requirements.txt

In [1]:
import torch
from pathlib import Path
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Using CPU")

CUDA available: True
GPU: NVIDIA L4
CUDA version: 12.1


In [ ]:
!python cnn_train/yolo/dataset_formatter.py

In [ ]:
!python cnn_train/yolo/yolo_train.py

In [ ]:
# Check if we have the required data structure
base_dir = Path('/teamspace/studios/this_studio')
train_dir = base_dir / 'downloaded_photos' / 'train'
val_dir = base_dir / 'downloaded_photos' / 'val'

if train_dir.exists():
    train_images = list(train_dir.glob('*.jpeg')) + list(train_dir.glob('*.jpg'))
    train_xmls = list(train_dir.glob('*.xml'))
    print(f"Training images: {len(train_images)}")
    print(f"Training XMLs: {len(train_xmls)}")

if val_dir.exists():
    val_images = list(val_dir.glob('*.jpeg')) + list(val_dir.glob('*.jpg'))
    val_xmls = list(val_dir.glob('*.xml'))
    print(f"Validation images: {len(val_images)}")
    print(f"Validation XMLs: {len(val_xmls)}")


# Run the weld extraction script
!python cnn_train/cnn/extract_welds.py

In [3]:
# Step 3: Train ResNet-50 Binary Classifier
!python cnn_train/cnn/welding_cnn.py --mode "train"

Using device: cuda
=== Training ResNet-50 Welding Classifier ===
Device: CUDA
Using train/validation split (no separate test set)

Training Configuration:
  epochs: 50
  batch_size: 32
  patience: 8
  save_dir: /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/runs

=== Starting Training ===
Found 8481 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/train/good
Found 2390 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/train/bad
Found 3749 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/val/good
Found 947 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/val/bad
Total parameters: 25,557,032
Trainable parameters before freezing: 25,557,032
Trainable parameters after freezing: 22,065,153
Model initialized with 22,065,153 trainable parameters

=== Starting Training ===
Epochs: 50, Batch Size: 32, Patience: 8

Epoch 1/50
Training:  15%|▏| 51/340 [00:16<01:17

In [4]:
!python cnn_train/cnn/welding_cnn.py --mode "val"

Using device: cuda
Total parameters: 25,557,032
Trainable parameters before freezing: 25,557,032
Model initialized with 23,510,081 trainable parameters
Found 3749 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/val/good
Found 947 images in /teamspace/studios/this_studio/cnn_model/cnn_train/cnn/extracted_welds/val/bad
Running evaluation on test data...
Evaluating: 100%|█████████████████████████████| 147/147 [00:24<00:00,  6.09it/s]

=== Evaluation Results ===
AUC-ROC Score: 0.9906
PR-AUC Score: 0.9801
F1 Score: 0.9727
Threshold used: 0.5

Classification Report:
              precision    recall  f1-score   support

        Good       0.99      1.00      0.99      3749
         Bad       0.98      0.96      0.97       947

    accuracy                           0.99      4696
   macro avg       0.99      0.98      0.98      4696
weighted avg       0.99      0.99      0.99      4696


Confusion Matrix:
Predicted:  Good  Bad
Good:       3735   14
Bad:      